In [ ]:
from glob import glob
from os.path import join
from tensorflow.keras import layers, models
import xarray as xr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import imports.GDL_layers as GDL_layers

In [ ]:
# Find a list of all the datafiles
patch_path = "/glade/scratch/lverhoef/WRF_all/track_data_hrrr_3km_nc_refl/"
patch_files = sorted(glob(join(patch_path, "*.nc")))
patch_ds = xr.open_dataset(patch_files[0])
csv_path = "/glade/scratch/lverhoef/WRF_all/track_data_hrrr_3km_csv_refl/"
csv_files = sorted(glob(join(csv_path, "track_step_*.csv")))
meta_ds = pd.read_csv(csv_files[0])

In [ ]:
patch_ds

In [ ]:
meta_ds

In [ ]:
# Pull selected variables from patch files and join into a single DataSet
num_files = 150
train_split = int(num_files*0.7)
val_split = int(num_files*0.8)
variables = ["REFL_COM_curr"]
data_list = []
for p, patch_file in enumerate(patch_files[0:train_split]):
    if p % 10 == 0:
        print(f'Train {p}, {patch_file}')
    ds = xr.open_dataset(patch_file)
    data_list.append(ds[variables].compute())
    ds.close()
input_train = xr.concat(data_list, dim="p")["REFL_COM_curr"].expand_dims("channel", axis = -1)
data_list = []
for p, patch_file in enumerate(patch_files[train_split:val_split]):
    if p % 10 == 0:
        print(f'Validation {train_split + p}, {patch_file}')
    ds = xr.open_dataset(patch_file)
    data_list.append(ds[variables].compute())
    ds.close()
input_val = xr.concat(data_list, dim="p")["REFL_COM_curr"].expand_dims("channel", axis = -1)
data_list = []
for p, patch_file in enumerate(patch_files[val_split:num_files]):
    if p % 10 == 0:
        print(f'Test {val_split + p}, {patch_file}')
    ds = xr.open_dataset(patch_file)
    data_list.append(ds[variables].compute())
    ds.close()
input_test = xr.concat(data_list, dim="p")["REFL_COM_curr"].expand_dims("channel", axis = -1)

In [ ]:
# Pull variables from csv files and join into an array
csv_variables = ["major_axis_length", "minor_axis_length"]
csv_data_list = []
for p, csv_file in enumerate(csv_files[0:train_split]):
    if p % 10 == 0:
        print(f'Train {p}, {csv_file}')
    csv_ds = pd.read_csv(csv_file)
    csv_data_list.append(csv_ds[csv_variables].to_xarray().rename({'index': 'p'}))
output_train = xr.concat(csv_data_list, dim="p").to_array().transpose()
csv_data_list = []
for p, csv_file in enumerate(csv_files[train_split:val_split]):
    if p % 10 == 0:
        print(f'Validation {train_split + p}, {csv_file}')
    csv_ds = pd.read_csv(csv_file)
    csv_data_list.append(csv_ds[csv_variables].to_xarray().rename({'index': 'p'}))
output_val = xr.concat(csv_data_list, dim="p").to_array().transpose()
csv_data_list = []
for p, csv_file in enumerate(csv_files[val_split:num_files]):
    if p % 10 == 0:
        print(f'Test {val_split + p}, {csv_file}')
    csv_ds = pd.read_csv(csv_file)
    csv_data_list.append(csv_ds[csv_variables].to_xarray().rename({'index': 'p'}))
output_test = xr.concat(csv_data_list, dim="p").to_array().transpose()

In [ ]:
# Normalize the training data
scale_stats = pd.DataFrame(index=[0], columns=["mean", "sd"])
scale_stats.loc[0, "mean"] = input_train.mean()
scale_stats.loc[0, "sd"] = input_train.std()
input_train_norm = (input_train - scale_stats.loc[0, "mean"]) / scale_stats.loc[0, "sd"]
input_val_norm = (input_val - scale_stats.loc[0, "mean"]) / scale_stats.loc[0, "sd"]
input_test_norm = (input_test - scale_stats.loc[0, "mean"]) / scale_stats.loc[0, "sd"]

In [ ]:
# Normalize output data
output_scale_stats = pd.DataFrame(index=[0], columns=["mean", "sd"])
output_scale_stats.loc[0, "mean"] = output_train.mean()
output_scale_stats.loc[0, "sd"] = output_train.std()
output_train_norm = (output_train - output_scale_stats.loc[0, "mean"]) / output_scale_stats.loc[0, "sd"]
output_val_norm = (output_val - output_scale_stats.loc[0, "mean"]) / output_scale_stats.loc[0, "sd"]
output_test_norm = (output_test - output_scale_stats.loc[0, "mean"]) / output_scale_stats.loc[0, "sd"]

In [ ]:
inv_model = models.Sequential()
inv_model.add(GDL_layers.RotEquivConv2D(32, (3, 3), rot_axis=False, input_shape=(144, 144, 1)))
inv_model.add(GDL_layers.RotEquivPool2D((2, 2)))
inv_model.add(GDL_layers.RotEquivConv2D(32, (3, 3)))
inv_model.add(GDL_layers.RotEquivPool2D((2, 2)))
inv_model.add(GDL_layers.RotEquivConv2D(64, (3, 3)))
inv_model.add(GDL_layers.RotEquivPool2D((2, 2)))
inv_model.add(GDL_layers.RotEquivConv2D(64, (3, 3)))
inv_model.add(GDL_layers.RotEquivPool2D((2, 2)))
inv_model.add(GDL_layers.RotEquivConv2D(128, (3, 3)))
inv_model.add(GDL_layers.RotInvPool())
inv_model.add(layers.Flatten())
inv_model.add(layers.Dense(32, activation='relu'))
inv_model.add(layers.Dense(2))

In [ ]:
inv_model.summary()

In [ ]:
inv_model.compile(
    optimizer='nadam',
    loss='mse',
    metrics=['mse']
)

In [ ]:
inv_history = inv_model.fit(x=input_train_norm, y=output_train_norm, epochs=20, validation_data=(input_val_norm, output_val_norm))

In [ ]:
equiv_model = models.Sequential()
equiv_model.add(GDL_layers.RotEquivConv2D(32, (3, 3), rot_axis=False, input_shape=(144, 144, 1)))
equiv_model.add(GDL_layers.RotEquivPool2D((2, 2)))
equiv_model.add(GDL_layers.RotEquivConv2D(32, (3, 3)))
equiv_model.add(GDL_layers.RotEquivPool2D((2, 2)))
equiv_model.add(GDL_layers.RotEquivConv2D(64, (3, 3)))
equiv_model.add(GDL_layers.RotEquivPool2D((2, 2)))
equiv_model.add(GDL_layers.RotEquivConv2D(64, (3, 3)))
equiv_model.add(GDL_layers.RotEquivPool2D((2, 2)))
equiv_model.add(GDL_layers.RotEquivConv2D(100, (3, 3)))
equiv_model.add(layers.Flatten())
equiv_model.add(layers.Dense(32, activation='relu'))
equiv_model.add(layers.Dense(2))

In [ ]:
equiv_model.summary()

In [ ]:
equiv_model.compile(
    optimizer='nadam',
    loss='mse',
    metrics=['mse']
)

In [ ]:
equiv_history = equiv_model.fit(x=input_train_norm, y=output_train_norm, epochs=20, validation_data=(input_val_norm, output_val_norm))

In [ ]:
cnn_model = models.Sequential()
cnn_model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(144, 144, 1)))
cnn_model.add(layers.MaxPooling2D((2, 2)))
cnn_model.add(layers.Conv2D(32, (3, 3), activation='relu'))
cnn_model.add(layers.MaxPooling2D((2, 2)))
cnn_model.add(layers.Conv2D(64, (3, 3), activation='relu'))
cnn_model.add(layers.MaxPooling2D((2, 2)))
cnn_model.add(layers.Conv2D(64, (3, 3), activation='relu'))
cnn_model.add(layers.MaxPooling2D((2, 2)))
cnn_model.add(layers.Conv2D(128, (3, 3), activation='relu'))
cnn_model.add(layers.Flatten())
cnn_model.add(layers.Dense(32, activation='relu'))
cnn_model.add(layers.Dense(2))

In [ ]:
cnn_model.summary()

In [ ]:
cnn_model.compile(
    optimizer='nadam',
    loss='mse',
    metrics=['mse']
)

In [ ]:
cnn_history = cnn_model.fit(x=input_train_norm, y=output_train_norm, epochs=20, validation_data=(input_val_norm, output_val_norm))

In [ ]:
F = plt.Figure(figsize=(6, 4))
plt.plot(cnn_history.history['mse'], label='CNN training', color='orange')
plt.plot(cnn_history.history['val_mse'], label='CNN validation', color='orange', linestyle='dashed')
plt.plot(inv_history.history['mse'], label='Inv training', color='blue')
plt.plot(inv_history.history['val_mse'], label='Inv validation', color='blue', linestyle='dashed')
plt.plot(equiv_history.history['mse'], label='Equiv training', color='green')
plt.plot(equiv_history.history['val_mse'], label='Equiv validation', color='green', linestyle='dashed')
plt.legend()
plt.title("MSE History")
ax = plt.gca()
ax.set_ylim(bottom=0)

In [ ]:
inv_model.save("saved_models/rot_equiv_vs_CNN/inv_model")
equiv_model.save("saved_models/rot_equiv_vs_CNN/equiv_model")
cnn_model.save("saved_models/rot_equiv_vs_CNN/cnn_model")

In [ ]:
inv_model.evaluate(input_test_norm, output_test_norm)

In [ ]:
equiv_model.evaluate(input_test_norm, output_test_norm)

In [ ]:
cnn_model.evaluate(input_test_norm, output_test_norm)